## **Labeling of satellite images based on the different atmospheric conditions and different terrain regions using CNN.**

### **Importing the libraries**

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import gc

## **Loading the datasets**

In [52]:
#reading the labels
train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [53]:
encoder = LabelEncoder()

In [54]:
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [55]:
#defining a dict of encoded labels

label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [56]:
#loading the traing_images

X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [58]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [02:10<00:00, 310.99it/s]


In [60]:
X = np.array(X, np.float16)/255.0
Y = np.array(Y)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(37838, 64, 64, 3) (37838, 17) (9460, 64, 64, 3) (9460, 17)


In [61]:
del X, Y
gc.collect()

989

In [62]:
gc.collect()


23

In [64]:
def fbeta(y_true, y_pred, beta=2, threshold=0.0):
    y_pred = K.clip(y_pred, min_value=0.0, max_value=1.0)
    y_pred_bin = K.round(y_pred + threshold)  
    tp = K.sum(K.cast(y_true * y_pred_bin, dtype='float32')) + K.epsilon()
    fp = K.sum(K.cast(K.clip(y_pred_bin - y_true, 0, 1), dtype='float32'))
    fn = K.sum(K.cast(K.clip(y_true - y_pred, 0, 1), dtype='float32'))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    beta_squared = beta ** 2
    fbeta_score = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return fbeta_score

## **Define the model**

In [66]:
model = Sequential()
model.add(Conv2D(64, 5, 2, activation="relu", input_shape=(64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(17, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=[fbeta])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Training model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=45, batch_size=64, callbacks=[early_stopping])

Epoch 1/45
592/592 [==============================] - 43s 72ms/step - loss: 0.2481 - fbeta: 0.6080 - val_loss: 0.2297 - val_fbeta: 0.6365
Epoch 2/45
592/592 [==============================] - 41s 70ms/step - loss: 0.2226 - fbeta: 0.6286 - val_loss: 0.2176 - val_fbeta: 0.6472
Epoch 3/45
592/592 [==============================] - 40s 68ms/step - loss: 0.2175 - fbeta: 0.6390 - val_loss: 0.2165 - val_fbeta: 0.6395
Epoch 4/45
592/592 [==============================] - 40s 68ms/step - loss: 0.2110 - fbeta: 0.6532 - val_loss: 0.2041 - val_fbeta: 0.6499
Epoch 5/45
592/592 [==============================] - 40s 68ms/step - loss: 0.2004 - fbeta: 0.6755 - val_loss: 0.1936 - val_fbeta: 0.6842
Epoch 6/45
592/592 [==============================] - 41s 70ms/step - loss: 0.1942 - fbeta: 0.6882 - val_loss: 0.1883 - val_fbeta: 0.6945
Epoch 7/45
592/592 [==============================] - 42s 71ms/step - loss: 0.1896 - fbeta: 0.6954 - val_loss: 0.1884 - val_fbeta: 0.6988
Epoch 8/45
592/592 [==============

In [67]:
gc.collect()

5273

In [68]:
test_loss, test_accuracy = model.evaluate(X_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

296/296 [==============================] - 4s 14ms/step - loss: 0.1404 - fbeta: 0.7850
Test loss: 0.140354722738266
Test accuracy: 0.7849506735801697


In [69]:
# Split the data into two parts
test_part1 = sam_sub.iloc[:40669] 
test_part2 = sam_sub.iloc[40669:]

# Load and preprocess images from the first directory
test_img_part1 = []
for image_name in tqdm(test_part1['image_name'], miniters=1000):
    arr = cv2.imread(f'../input/planets-dataset/planet/planet/test-jpg/{image_name}.jpg', cv2.IMREAD_UNCHANGED)
    arr = cv2.resize(arr, (64, 64))
    test_img_part1.append(arr)

# Load and preprocess images from the second directory
test_img_part2 = []
for image_name in tqdm(test_part2['image_name'], miniters=1000):
    arr = cv2.imread(f'../input/planets-dataset/test-jpg-additional/test-jpg-additional/{image_name}.jpg', cv2.IMREAD_UNCHANGED)
    arr = cv2.resize(arr, (64, 64))
    test_img_part2.append(arr)


100%|██████████| 20522/20522 [01:24<00:00, 243.02it/s]


In [71]:
# Concatenate the two parts of images
test_img = np.concatenate([test_img_part1, test_img_part2], axis=0)

# Normalize the images
test_img = test_img.astype(np.float16) / 255.0

# Predict on the test set
predictions = model.predict(test_img)

# Convert predictions to tags
threshold = 0.0 
preds = []

# Iterate through each prediction
for pred in predictions:
    # Get the indices where the prediction is above the threshold
    indices = [i for i, p in enumerate(pred) if p > threshold]
    
    # Map the indices to their corresponding tags using label_maps
    tags = [label_maps.iloc[i]['tags'] for i in indices]
    
    # Join the tags into a single string
    preds.append(' '.join(tags))

In [72]:
# Create submission file
submission = pd.DataFrame({'image_name': sam_sub['image_name'], 'tags': preds})
submission.to_csv('submission.csv', index=False)
